## Try-on Clothes: Image Difference Mask Notebook

This notebook replicates the functionality of `basic.py` but adds interactive controls:

- Select the two input images from `inputs/`
- Tune blur, threshold, morphology, and edge trim parameters
- Preview the gray diff and the final mask inline
- Save outputs to `outputs/` with timestamped filenames to avoid overwriting

Tip: Put your two images (original and edited) into the `inputs/` folder before starting.

In [1]:
# Imports
import os, glob
from datetime import datetime
import cv2
import numpy as np
from IPython.display import display
import ipywidgets as widgets
from pathlib import Path

# Assume current working directory is the notebook folder
BASE_DIR = Path.cwd()  # notebook-image-manip
INPUT_DIR = BASE_DIR / 'inputs'
OUTPUT_DIR = BASE_DIR / 'outputs'

OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

def list_images():
    exts = ('.png','.jpg','.jpeg','.bmp','.tif','.tiff')
    if not INPUT_DIR.exists():
        return []
    return sorted([p.name for p in INPUT_DIR.iterdir() if p.suffix.lower() in exts])

image_files = list_images()
if not image_files:
    print(f'No images found in {INPUT_DIR}. Please add original and edited images.')
else:
    print(f'Found {len(image_files)} image(s): {image_files}')

Found 3 image(s): ['me.png', 'pjs_2.jpg', 'tryon_result.jpg']


In [ ]:
# Widgets for selecting inputs and tuning parameters
if image_files:
    orig_dropdown = widgets.Dropdown(options=image_files, description='Original:')
    edited_dropdown = widgets.Dropdown(options=image_files, description='Edited:')
else:
    orig_dropdown = widgets.Dropdown(options=[''], description='Original:')
    edited_dropdown = widgets.Dropdown(options=[''], description='Edited:')

def _odd(n: int) -> int:
    # make sure kernel sizes are odd and >=1
    n = max(1, int(n))
    return n if n % 2 == 1 else n + 1

blur_k = widgets.IntSlider(value=5, min=1, max=51, step=2, description='Blur k:', continuous_update=False)
thresh_v = widgets.IntSlider(value=10, min=0, max=255, step=1, description='Threshold:', continuous_update=False)
morph_k = widgets.IntSlider(value=9, min=1, max=51, step=2, description='Morph k:', continuous_update=False)
edge_pad = widgets.IntSlider(value=15, min=0, max=200, step=1, description='Edge pad:', continuous_update=False)
show_intermediate = widgets.Checkbox(value=False, description='Show diff gray')
auto_resize = widgets.Checkbox(value=False, description='Auto-resize edited to original')

controls = widgets.VBox([
    widgets.HBox([orig_dropdown, edited_dropdown]),
    widgets.HBox([blur_k, thresh_v]),
    widgets.HBox([morph_k, edge_pad]),
    widgets.HBox([show_intermediate, auto_resize]),
])

# controls

In [4]:
# Processing and interactive display
import matplotlib.pyplot as plt
from matplotlib import gridspec

out_area = widgets.Output()
button_row = widgets.HBox([])  # save button row placed under the images

# cache last results to enable saving without reprocessing
last_gray = None
last_mask = None

def process_and_preview(orig_name, edited_name, blur_k, thresh_v, morph_k, edge_pad, show_intermediate=False, auto_resize=False):
    global last_gray, last_mask
    out_area.clear_output(wait=True)
    with out_area:
        if not orig_name or not edited_name or orig_name == edited_name:
            print('Select two different images from inputs/.')
            last_gray = None; last_mask = None
            return None
        orig_path = str(INPUT_DIR / orig_name)
        edit_path = str(INPUT_DIR / edited_name)
        orig = cv2.imread(orig_path)
        edit = cv2.imread(edit_path)
        if orig is None:
            print(f'Could not read original image at {orig_path}')
            last_gray = None; last_mask = None
            return None
        if edit is None:
            print(f'Could not read edited image at {edit_path}')
            last_gray = None; last_mask = None
            return None
        if orig.shape[:2] != edit.shape[:2]:
            if auto_resize:
                edit = cv2.resize(edit, (orig.shape[1], orig.shape[0]), interpolation=cv2.INTER_LINEAR)
            else:
                print(f'Images must be same size, got orig={orig.shape[:2]} edited={edit.shape[:2]}')
                last_gray = None; last_mask = None
                return None

        k_b = _odd(blur_k)
        k_m = _odd(morph_k)
        orig_b = cv2.GaussianBlur(orig, (k_b, k_b), 0)
        edit_b = cv2.GaussianBlur(edit, (k_b, k_b), 0)

        diff = cv2.absdiff(orig_b, edit_b)
        gray = cv2.cvtColor(diff, cv2.COLOR_BGR2GRAY)
        _, mask = cv2.threshold(gray, int(thresh_v), 255, cv2.THRESH_BINARY)

        kernel = np.ones((k_m, k_m), np.uint8)
        mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
        mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)

        pad = int(max(0, edge_pad))
        h, w = mask.shape[:2]
        if pad > 0:
            mask[:pad, :] = 0
            mask[h-pad:, :] = 0
            mask[:, :pad] = 0
            mask[:, w-pad:] = 0

        # Single-row layout: Original | Edited | (Diff gray optional) | Mask
        n_cols = 4 if show_intermediate else 3
        fig, axes = plt.subplots(1, n_cols, figsize=(12, 12))
        plt.subplots_adjust(wspace=0.05)  # tighter horizontal space

        ax_idx = 0
        axes[ax_idx].imshow(cv2.cvtColor(orig, cv2.COLOR_BGR2RGB))
        axes[ax_idx].set_title('Original')
        axes[ax_idx].axis('off')
        ax_idx += 1
        axes[ax_idx].imshow(cv2.cvtColor(edit, cv2.COLOR_BGR2RGB))
        axes[ax_idx].set_title('Edited')
        axes[ax_idx].axis('off')
        ax_idx += 1
        if show_intermediate:
            axes[ax_idx].imshow(gray, cmap='gray')
            axes[ax_idx].set_title('Diff gray')
            axes[ax_idx].axis('off')
            ax_idx += 1
        axes[ax_idx].imshow(mask, cmap='gray')
        axes[ax_idx].set_title('Mask')
        axes[ax_idx].axis('off')
        fig.tight_layout()
        plt.show()

        # update cache
        last_gray, last_mask = gray, mask

        return gray, mask

ui = widgets.interactive_output(process_and_preview, {'orig_name': orig_dropdown,
                                                        'edited_name': edited_dropdown,
                                                        'blur_k': blur_k,
                                                        'thresh_v': thresh_v,
                                                        'morph_k': morph_k,
                                                        'edge_pad': edge_pad,
                                                        'show_intermediate': show_intermediate,
                                                        'auto_resize': auto_resize})

display(controls)
display(out_area)
display(button_row)  # save button row directly below images
display(ui)

Output()

HBox()

Output()

In [5]:
# Save current outputs to timestamped files (uses cached last_gray/last_mask)
last_saved_label = widgets.HTML(value='')
save_btn = widgets.Button(description='Save outputs', button_style='success', icon='save')

def save_current_outputs(_=None):
    global last_gray, last_mask
    if last_gray is None or last_mask is None:
        last_saved_label.value = '<em>Nothing to save. Generate a mask first.</em>'
        return
    ts = datetime.now().strftime('%Y%m%d_%H%M%S')
    gray_path = OUTPUT_DIR / f'diff_gray_{ts}.png'
    mask_path = OUTPUT_DIR / f'clothes_mask_{ts}.png'
    cv2.imwrite(str(gray_path), last_gray)
    cv2.imwrite(str(mask_path), last_mask)
    last_saved_label.value = f'Saved:<br>{gray_path.name}<br>{mask_path.name}'

save_btn.on_click(save_current_outputs)
button_row.children = [save_btn, last_saved_label]

button_row

In [ ]:
# (Optional) Test save programmatically
if len(image_files) >= 2:
    orig_dropdown.value = image_files[0]
    edited_dropdown.value = image_files[1]
save_current_outputs(None)
print('Saved files now in outputs/:', sorted([p.name for p in OUTPUT_DIR.iterdir()]))